In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


### 1) Importing libraries and dataset

In [ ]:
# path = '/content/drive/MyDrive/nlp/Aggression_Imbalance_DS.csv'

In [1]:
!nvidia-smi

Sat Jul  9 17:00:06 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# !pip install transformers
# !pip install datasets
# !pip install 'ray[tune]'

: 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoTokenizer
from transformers import Trainer, TrainingArguments
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split

/home/diptesh/anaconda3/envs/aggDet/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# either put path(if defined) in train value here

aggressions = load_dataset('csv', data_files={'train': 'Aggression_Imbalance_DS.csv'})

aggressions.set_format(type='pandas') # Converting from HF's dataset type to pandas dataframe
df = aggressions['train'][:]
df.head()

Using custom data configuration default-049c89cbaba4ec71


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 509.88it/s]


Dataset csv downloaded and prepared to /home/diptesh/.cache/huggingface/datasets/csv/default-049c89cbaba4ec71/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 323.56it/s]


,Sentence,Category
0,what a wicket !!!!!!!!! woohooooo #indvsuae #w...,NAG
1,And because of u people there is a boundary cr...,OAG
2,Arandhuti Roy is a person who do not believe i...,NAG
3,Saurabh Shinde wht does that mean at least he ...,CAG
4,NBCC has a very good order book but will that ...,NAG


In [3]:
df['Category'].value_counts()

NAG    5602
CAG    4240
OAG    2821
Name: Category, dtype: int64

In [4]:
# Label encoding

dt = {'NAG':0, 'CAG':1, 'OAG':2}
df['Category'].unique()
labels = map(lambda x: dt[x], df['Category'])
labels = list(labels)
labels = pd.Series(labels)

df['Label'] = labels
labs = np.array(labels)
df.head(5)

,Sentence,Category,Label
0,what a wicket !!!!!!!!! woohooooo #indvsuae #w...,NAG,0
1,And because of u people there is a boundary cr...,OAG,2
2,Arandhuti Roy is a person who do not believe i...,NAG,0
3,Saurabh Shinde wht does that mean at least he ...,CAG,1
4,NBCC has a very good order book but will that ...,NAG,0


### 2) Tokenizing and making train, test and valid splits

In [5]:
model_ckpt = 'roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [6]:
texts = list(df['Sentence'])
labels = list(df['Label'])

In [7]:
train_size=0.8

# In the first step we will split the data in training and remaining dataset
train_texts, X_rem, train_labels, y_rem = train_test_split(texts, labels, train_size=0.8, random_state=43)

# Now since we want the valid and test size to be equal (10% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data)
test_size = 0.5
valid_texts, test_texts, valid_labels, test_labels = train_test_split(X_rem, y_rem, test_size=0.5, random_state=43)

print(str(np.shape(train_texts))), print(str(np.shape(train_labels)))
print(str(np.shape(valid_texts))), print(str(np.shape(valid_labels)))
print(str(np.shape(test_texts))), print(str(np.shape(test_labels)))

(10130,)
(10130,)
(1266,)
(1266,)
(1267,)
(1267,)


(None, None)

In [8]:
# USE this only for hyperparameter tuning
# In this step we will split the train data (15 %) and validation data (50%) only for hyperparameter tuning 


_texts, hyp_texts, _labels, hyp_labels = train_test_split(train_texts, train_labels, test_size=0.15, random_state=43)
print(str(np.shape(hyp_texts))), print(str(np.shape(hyp_labels)))

_texts2, hyp_v_texts, _labels2, hyp_v_labels = train_test_split(valid_texts, valid_labels, test_size=0.5, random_state=43)
print(str(np.shape(hyp_v_texts))), print(str(np.shape(hyp_v_labels)))



(1520,)
(1520,)
(633,)
(633,)


(None, None)

In [9]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
valid_encodings = tokenizer(valid_texts, truncation=True, padding=True)
hyp_encodings = tokenizer(hyp_texts, truncation=True, padding=True)
hyp_v_encodings = tokenizer(hyp_v_texts, truncation=True, padding=True)

In [10]:
import torch
# tensors
class AggressionDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [11]:
train_dataset = AggressionDataset(train_encodings, train_labels)
valid_dataset = AggressionDataset(valid_encodings, valid_labels)
hyp_dataset = AggressionDataset(hyp_encodings, hyp_labels)
hyp_v_dataset = AggressionDataset(hyp_v_encodings, hyp_v_labels)

In [12]:
# Run for out of memory cases

# import gc

# gc.collect()
# torch.cuda.empty_cache()

### 3) Custom Trainer with weighted loss objective

In [13]:
from torch import nn

class CustomTrainer(Trainer):
  def compute_loss(self, model, inputs, return_outputs=False):
    device = torch.device('cuda')
    inputs.to(device)
    labels = inputs.get("labels")
    # forward pass
    outputs = model(**inputs)
    logits = outputs.get("logits")
    # compute custom loss (suppose one has 3 labels with different weights)
    loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([0.21, 0.33, 0.46]).to(device))
    loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
    return (loss, outputs) if return_outputs else loss

### 4) Hyperparameter Tuning (using PBT)

In [14]:
from ray.tune.schedulers import PopulationBasedTraining
import ray
from ray import tune
from ray.tune import CLIReporter

In [15]:
from transformers import AutoModelForSequenceClassification

# num_labels = 3
# device = torch.device('cuda')
# if torch.cuda.is_available() else 'cpu'
# print(device)
# model = (AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels).to(device))

def get_my_model():
  num_labels = 3
  device = torch.device('cuda')
  return (AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels).to(device))

In [16]:
from datasets import load_metric

def compute_metrics(pred):
  labels = pred.label_ids
  preds = pred.predictions.argmax(-1)
  metric = load_metric("f1")
  return metric.compute(predictions=preds, references=labels, average="macro")

In [22]:
# See this for more stoppers
# https://docs.ray.io/en/latest/tune/api_docs/stoppers.html

from ray.tune import Stopper

class CustomStopper(tune.Stopper):
        def __init__(self):
            self.should_stop = False

        def __call__(self, trial_id, result):
            max_iter = 5 
            if not self.should_stop and result["training_iteration"] >= max_iter:
                self.should_stop = True

        def stop_all(self):
            return self.should_stop

stopper = CustomStopper()

In [26]:
from transformers import Trainer, TrainingArguments
from ray.tune.examples.pbt_transformers.utils import build_compute_metrics_fn

# task_name = 'rte'

training_args = TrainingArguments(
        output_dir=".",
        learning_rate=1.5e-5,  # config
        do_train=True,
        do_eval=True,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        num_train_epochs=2,  # config
        max_steps=-1,
        per_device_train_batch_size=8,  # config
        per_device_eval_batch_size=8,  # config
        warmup_steps=0,
        weight_decay=0.1,  # config
        logging_dir="./logs",
        skip_memory_metrics=True,
        report_to="none",
        # metric_for_best_model=metric_name,

    )

trainer = CustomTrainer(model_init=get_my_model, args=training_args,
        compute_metrics = compute_metrics,
        train_dataset = hyp_dataset,
        eval_dataset = hyp_v_dataset,
        tokenizer = tokenizer)

tune_config = {
        "per_device_train_batch_size": tune.choice([8]),
        "per_device_eval_batch_size": 8,
        "num_train_epochs": tune.choice([3, 4, 5, 6]),
        # "max_steps": 1 if smoke_test else -1,  # Used for smoke test.
    }

scheduler = PopulationBasedTraining(
        time_attr="training_iteration",
        metric="eval_f1",
        mode="max",
        perturbation_interval=1,
        hyperparam_mutations={
            "weight_decay": tune.uniform(0.01, 0.3),
            "learning_rate": tune.uniform(1e-5, 5e-5),
            "per_device_train_batch_size": [8],
        },
    )

reporter = CLIReporter(
        parameter_columns={
            "weight_decay": "w_decay",
            "learning_rate": "lr",
            "per_device_train_batch_size": "train_bs/gpu",
            "num_train_epochs": "num_epochs",
        },
        metric_columns=["eval_f1", "eval_loss", "epoch", "training_iteration"],
    )


ress = trainer.hyperparameter_search(
        hp_space=lambda _: tune_config,
        backend="ray",
        n_trials=4,
        resources_per_trial={"cpu": 2, "gpu": 1}, # can be parallelized , https://docs.ray.io/en/latest/tune/tutorials/tune-resources.html
        scheduler=scheduler,
        keep_checkpoints_num=1,
        checkpoint_score_attr="training_iteration",
        stop=stopper,
        progress_reporter=reporter,
        local_dir="./ray_results/",
        name="tune_transformer_pbt",
        log_to_file=True,
    )

PyTorch: setting up devices
loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /home/diptesh/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_emb

== Status ==
Current time: 2022-07-12 17:49:18 (running for 00:00:00.19)
Memory usage on this node: 23.5/125.5 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 2.0/36 CPUs, 1.0/2 GPUs, 0.0/67.58 GiB heap, 0.0/32.95 GiB objects
Result logdir: /home/diptesh/workspace/AggressionDetection-IIITL/hyperparamTuning/ray_results/tune_transformer_pbt
Number of trials: 4/4 (3 PENDING, 1 RUNNING)
+------------------------+----------+-------------------------+-----------+-------------+----------------+--------------+
| Trial name             | status   | loc                     |   w_decay |          lr |   train_bs/gpu |   num_epochs |
|------------------------+----------+-------------------------+-----------+-------------+----------------+--------------|
| _objective_92162_00000 | RUNNING  | 131.227.165.153:2163234 | 0.240997  | 1.73374e-05 |              8 |            5 |
| _objective_92162_00001 | PENDING  |                         | 0.183611  | 1.62407e-05 |         

(_objective pid=2163234) Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight']
(_objective pid=2163234) - This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(_objective pid=2163234) - This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(_objective pid=2163234) Some weights of RobertaForSequenceClassification were not initialized from th

Result for _objective_92162_00000:
  date: 2022-07-12_17-49-21
  experiment_id: e165cd363ed940a5aa5dc5e36557e496
  hostname: maverick
  node_ip: 131.227.165.153
  pid: 2163234
  timestamp: 1657644561
  trial_id: '92162_00000'
  


(_objective pid=2163276) Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias']
(_objective pid=2163276) - This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(_objective pid=2163276) - This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(_objective pid=2163276) Some weights of RobertaForSequenceClassification were not initialized from th

...

(_objective pid=2163361) Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias']
(_objective pid=2163361) - This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(_objective pid=2163361) - This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(_objective pid=2163361) Some weights of RobertaForSequenceClassification were not initialized from th

Result for _objective_92162_00002:
  date: 2022-07-12_17-49-31
  experiment_id: f0cd781038ad415a99b9314e5b8988a1
  hostname: maverick
  node_ip: 131.227.165.153
  pid: 2163361
  timestamp: 1657644571
  trial_id: '92162_00002'
  
== Status ==
Current time: 2022-07-12 17:49:38 (running for 00:00:20.67)
Memory usage on this node: 31.5/125.5 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 2.0/36 CPUs, 1.0/2 GPUs, 0.0/67.58 GiB heap, 0.0/32.95 GiB objects
Result logdir: /home/diptesh/workspace/AggressionDetection-IIITL/hyperparamTuning/ray_results/tune_transformer_pbt
Number of trials: 4/4 (3 ERROR, 1 RUNNING)
+------------------------+----------+-------------------------+-----------+-------------+----------------+--------------+
| Trial name             | status   | loc                     |   w_decay |          lr |   train_bs/gpu |   num_epochs |
|------------------------+----------+-------------------------+-----------+-------------+----------------+---------

(_objective pid=2163402) Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.bias']
(_objective pid=2163402) - This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
(_objective pid=2163402) - This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
(_objective pid=2163402) Some weights of RobertaForSequenceClassification were not initialized from th

Result for _objective_92162_00003:
  date: 2022-07-12_17-49-34
  experiment_id: 39848859e4ef4303bd5242473f1fe433
  hostname: maverick
  node_ip: 131.227.165.153
  pid: 2163402
  timestamp: 1657644574
  trial_id: '92162_00003'
  
== Status ==
Current time: 2022-07-12 17:49:42 (running for 00:00:23.82)
Memory usage on this node: 28.0/125.5 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 0/36 CPUs, 0/2 GPUs, 0.0/67.58 GiB heap, 0.0/32.95 GiB objects
Result logdir: /home/diptesh/workspace/AggressionDetection-IIITL/hyperparamTuning/ray_results/tune_transformer_pbt
Number of trials: 4/4 (4 ERROR)
+------------------------+----------+-------------------------+-----------+-------------+----------------+--------------+
| Trial name             | status   | loc                     |   w_decay |          lr |   train_bs/gpu |   num_epochs |
|------------------------+----------+-------------------------+-----------+-------------+----------------+--------------|
| _objec

TuneError: ('Trials did not complete', [_objective_92162_00000, _objective_92162_00001, _objective_92162_00002, _objective_92162_00003])